In [1]:
from scripts.utilities import pltTimestamps, FEDtoReadOut
from scripts.plotting import plot_table, plot_box
from scripts.calcF_timeBased import get_bckg_frac

Welcome to JupyROOT 6.26/10


In [2]:
import math
import os
import pathlib
import sys
import argparse
import subprocess
from glob import glob
import pandas as pd

In [3]:
HOSTNAME = os.uname()[1]
if HOSTNAME == "DELLNK":
    sys.path.insert(0, '/home/nimmitha/utils')
elif "lxplus" in HOSTNAME or "scx5-c2f06-36" in HOSTNAME:
    sys.path.insert(0, '/afs/cern.ch/user/n/nkarunar/utils')
else:
    print("Hostname not recognized. Exiting.")
    sys.exit()
from nf import post_to_slack

In [4]:
PLT_PATH = os.getcwd().split("PLTOffline")[0] + "PLTOffline/"
FILE_PATH = "/home/nkarunar/track_root_files/"
FILE_EXT = ".root"

In [6]:
sys.argv = ['--list', '8730', '8741', '9050', '9059', '9070', '9072']
           # 8739 8741 9059 9070 9072']
parser = argparse.ArgumentParser(description='Calculate background fraction')
parser.add_argument('--list', nargs='+', type=int, help='List of fills to run on')
args = parser.parse_args(sys.argv)

In [7]:
channels = [i for i in range(0, 16)]
# drop channel 6
channels.remove(6)

In [8]:
def get_inst_luminosity(df_row):

    def parseDateBrilcalc(x): return pd.to_datetime(x, format='%m/%d/%y %H:%M:%S')

    fill = str(df_row.name)
    fName = str(fill)
    tempLumifPath = "temp/" + fName + "tempLumi.csv"
    print(f"Running brilcalc for {fill}")

    hh, mm = df_row['start'].strftime("%H:%M").split(":")
    StartTime = str(3600 * int(hh) + 60 * int(mm))

    hh, mm = df_row['end'].strftime("%H:%M").split(":")
    EndTime = str(3600 * int(hh) + 60 * int(mm))

    cmd_brilcalc = ["brilcalc", "lumi", "--begin", "mm/dd/yy hh:mm:ss", "--end",
                    "mm/dd/yy hh:mm:ss", "--byls", "-u", "hz/ub", "--type", "PLTZERO", "-o", tempLumifPath]

    cmd_brilcalc[3] = df_row['start'].strftime("%m/%d/%y %H:%M:%S")
    cmd_brilcalc[5] = df_row['end'].strftime("%m/%d/%y %H:%M:%S")

    # print(cmd_brilcalc)
    x = subprocess.run(cmd_brilcalc)
    print(x)

    df_lumi = pd.read_csv(tempLumifPath, comment="#", header=None, names=[
        "fill", "ls", "time", "status", "E", "del", "rec", "avgpu", "source"], parse_dates=["time"], date_parser=parseDateBrilcalc)
    # print(df_lumi.head())

    StartTime = df_row['start']
    EndTime = df_row['end']
    step = pd.Timedelta(seconds=df_row['duration'])

    t1 = StartTime
    h_max = int((EndTime - StartTime) / step)

    lumiLogfPath = "logs/" + fName + "Lumi.csv"
    lumiLogFile = open(lumiLogfPath, 'w')
    lumiLogFile.write("fill,h,t1,t2,mean_lumi,mean_sbil\n")

    collidingBunches = 1

    for i in range(h_max):
        t1 = StartTime + i * step
        t2 = StartTime + (i + 1) * step

        mean_lumi = df_lumi[(df_lumi["time"] > t1) & (
            df_lumi["time"] <= t2)]["del"].mean()
        mean_sbil = mean_lumi / collidingBunches

        t1_save = t1.strftime('%s')
        t2_save = int(t2.strftime('%s')) - 1

        lineToWrite = fill + "," + str(i) + "," + str(t1_save) + "," + str(
            t2_save) + "," + str(mean_lumi) + "," + str(mean_sbil) + "\n"
        lumiLogFile.write(lineToWrite)

def run_fit_scripts(df_row):
    fill = str(df_row.name)
    print(f"Running bkg calculation for {fill}")

    StartTime = (int(df_row['start'].strftime("%s")) + 0 * 3600)
    EndTime = (int(df_row['end'].strftime("%s")) + 0 * 3600)
    step = (df_row['duration'])

    get_bckg_frac(fill, StartTime, EndTime, step, channels)


def add_timestamps_to_fills(pltTS, df):
    start_stable = []
    end_stable = []
    for index in df.index:
        start_date = pltTS.loc[index, "start_stable_beam"]
        end_date = pltTS.loc[index, "end_stable_beam"]

        start_stable.append((start_date + pd.Timedelta(minutes=5)).round(freq="5T"))
        end_stable.append((end_date - pd.Timedelta(minutes=5)).round(freq="5T"))

    df["start"] = start_stable
    df["end"] = end_stable

    df = df[['start', 'end', 'duration']]

    return df

def generate_missing_fills(pltTS):
    """
    Check the results folder and compare it with the decoded fills
    If there are newly decoded fills, create a list of them.
    This can also exclude some fills from the list
    """
    result_files = glob(os.path.join(PLT_PATH, 'BackgroundFraction/results', '????.csv'))
    result_files = [int(pathlib.Path(file).stem) for file in result_files]
    # print(result_files)
    decoded_files = glob(os.path.join(FILE_PATH, '????.root'))
    decoded_files = [int(pathlib.Path(file).stem) for file in decoded_files]
    # print(decoded_files)

    # Excluded fills
    excluded = [8178, 8225, 8381, 8385, 8999, 8957]
    missing_results = [fill for fill in decoded_files if fill not in result_files and fill not in excluded]

    return missing_results

def get_fill_df(args):
    pltTS = pltTimestamps(PLT_PATH)

    if args.list:
        fills_to_run = args.list
    else:
        fills_to_run = generate_missing_fills(pltTS)

    # print(fills_to_run)
    post_to_slack(message_text=f'Bkg will work on: {fills_to_run}')

    # Create a csv file with flls to run
    df = pd.DataFrame(columns=['start', 'end', 'duration'], index=fills_to_run)
    df.index.name = 'fill'

    df.duration = 300
    df = add_timestamps_to_fills(pltTS, df)

    df.to_csv("input_fills.csv", encoding='utf-8', header=True)
    return df


def combineLogs(df_row):
    
    def getCB(fill):
        df = pd.read_csv('https://delannoy.web.cern.ch/fills.csv')
        CB = int(df[(df['oms_fill_number'] == fill) & (df['oms_stable_beams']==True)]['oms_bunches_colliding'])
        return CB


    fill = str(df_row.name)
    CB = getCB(int(fill))
    fName = str(fill)
    lumiLogPath = "logs/" + fName + "Lumi.csv"
    fLogPath = "logs/" + fName + "F.csv"
    resultPath = "output/results/" + fName + ".csv"
    print(f"Combining logs for {fill}")

    # Read luminosity and Background Fraction logs
    lumi_df = pd.read_csv(lumiLogPath, index_col="h")
    f_df = pd.read_csv(fLogPath, index_col="h")

    if lumi_df.shape[0]*len(channels) != f_df.shape[0]:
        print("Lengths do not match")
        return

    result = pd.merge(lumi_df, f_df)
    # result["fSlopeY(%)"] = result.apply(lambda x: x.bkg_frac * 100, axis=1)
    # result["fSlopeY_e(%)"] = result.apply(lambda x: x.bkg_frac_e * 100, axis=1)
    # result["fR(%)"] = result.apply(lambda x: 100 * (x.ntracks_time - x.ntracks_resi) / x.ntracks_time, axis=1)
    # result["fR_e(%)"] = result.apply(lambda x: 100 * math.sqrt(x.ntracks_time - x.ntracks_resi) / x.ntracks_time, axis=1)
    result["mean_sbil"] = result.apply(lambda x: x.mean_sbil / CB, axis=1)
    result.to_csv(resultPath, encoding='utf-8', index=False)



In [9]:
pltTS = pltTimestamps(PLT_PATH)
fills_to_run = get_fill_df(args)
print(fills_to_run)

                   start                 end  duration
fill                                                  
8730 2023-05-07 07:30:00 2023-05-07 18:35:00       300
8741 2023-05-10 19:30:00 2023-05-11 02:55:00       300
9050 2023-07-10 12:35:00 2023-07-10 18:05:00       300
9059 2023-07-12 00:00:00 2023-07-12 01:55:00       300
9070 2023-07-15 17:25:00 2023-07-16 04:50:00       300
9072 2023-07-16 08:05:00 2023-07-16 20:25:00       300


In [10]:
for fill, row in fills_to_run.iterrows():
    print(f"Begin bkg calc {fill}")

    run_fit_scripts(row)
    get_inst_luminosity(row)
    combineLogs(row)
    # pass

Begin bkg calc 8730
Running bkg calculation for 8730
Processing 8730 from 1683437400 to 1683477300


100%|██████████| 133/133 [01:53<00:00,  1.18it/s]


Running brilcalc for 8730
CompletedProcess(args=['brilcalc', 'lumi', '--begin', '05/07/23 07:30:00', '--end', '05/07/23 18:35:00', '--byls', '-u', 'hz/ub', '--type', 'PLTZERO', '-o', 'temp/8730tempLumi.csv'], returncode=0)
Combining logs for 8730
Begin bkg calc 8741
Running bkg calculation for 8741
Processing 8741 from 1683739800 to 1683766500


100%|██████████| 89/89 [02:06<00:00,  1.42s/it]


Running brilcalc for 8741
CompletedProcess(args=['brilcalc', 'lumi', '--begin', '05/10/23 19:30:00', '--end', '05/11/23 02:55:00', '--byls', '-u', 'hz/ub', '--type', 'PLTZERO', '-o', 'temp/8741tempLumi.csv'], returncode=0)
Combining logs for 8741
Begin bkg calc 9050
Running bkg calculation for 9050
Processing 9050 from 1688985300 to 1689005100


...

Running brilcalc for 9059
CompletedProcess(args=['brilcalc', 'lumi', '--begin', '07/12/23 00:00:00', '--end', '07/12/23 01:55:00', '--byls', '-u', 'hz/ub', '--type', 'PLTZERO', '-o', 'temp/9059tempLumi.csv'], returncode=0)
Combining logs for 9059
Begin bkg calc 9070
Running bkg calculation for 9070
Processing 9070 from 1689434700 to 1689475800


100%|██████████| 137/137 [04:13<00:00,  1.85s/it]


Running brilcalc for 9070
CompletedProcess(args=['brilcalc', 'lumi', '--begin', '07/15/23 17:25:00', '--end', '07/16/23 04:50:00', '--byls', '-u', 'hz/ub', '--type', 'PLTZERO', '-o', 'temp/9070tempLumi.csv'], returncode=0)
Combining logs for 9070
Begin bkg calc 9072
Running bkg calculation for 9072
Processing 9072 from 1689487500 to 1689531900


100%|██████████| 148/148 [04:32<00:00,  1.84s/it]


Running brilcalc for 9072
CompletedProcess(args=['brilcalc', 'lumi', '--begin', '07/16/23 08:05:00', '--end', '07/16/23 20:25:00', '--byls', '-u', 'hz/ub', '--type', 'PLTZERO', '-o', 'temp/9072tempLumi.csv'], returncode=0)
Combining logs for 9072
